In [1]:
import os

In [2]:
%pwd

'd:\\Study\\Python\\ML\\car_price_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Study\\Python\\ML\\car_price_prediction'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

@dataclass
class PreprocessingConfig:
    target_column: str
    test_size: float
    random_state: int

In [10]:
from car_price_prediction.constants import *
from car_price_prediction.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_preprocessing_config(self) -> PreprocessingConfig:
        preprocessing = self.config.preprocessing
        params = self.params

        preprocessing_config = PreprocessingConfig(
            target_column=preprocessing.target_column,
            test_size=params.TEST_SIZE,
            random_state=params.RANDOM_STATE
        )

        return preprocessing_config
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_learning_rate=self.params.LEARNING_RATE
        )

        return prepare_base_model_config


NameError: name 'DataIngestionConfig' is not defined

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2025-07-12 10:32:05,751: INFO: font_manager: generated new fontManager]


In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from pathlib import Path
import pandas as pd
from car_price_prediction import logger

class PrepareLinearModel:
    def __init__(self, config):
        self.config = config
        self.model = LinearRegression()

    def train_model(self, X_train, y_train):
        """
        Train the Linear Regression model and save it.
        """
        self.model.fit(X_train, y_train)
        logger.info("Linear Regression model trained successfully.")
        self.save_model(path=self.config.model_path, model=self.model)

    def evaluate_model(self, X_test, y_test):
        """
        Evaluate the model using MSE and R2 score.
        """
        predictions = self.model.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        logger.info(f"Model Evaluation - MSE: {mse:.4f}, R2 Score: {r2:.4f}")
        return mse, r2

    @staticmethod
    def save_model(path: Path, model: LinearRegression):
        """
        Save the trained model to disk.
        """
        joblib.dump(model, path)
        logger.info(f"Model saved at: {path}")

    @staticmethod
    def load_model(path: Path) -> LinearRegression:
        """
        Load a saved model from disk.
        """
        model = joblib.load(path)
        logger.info(f"Model loaded from: {path}")
        return model

In [19]:
import pandas as pd

try:
    # Configuration setup
    config = ConfigurationManager()

    # Load dataset
    data_ingestion_config = config.get_data_ingestion_config()
    df = pd.read_csv(data_ingestion_config.local_data_file)

    # Preprocessing
    preprocessing_config = config.get_preprocessing_config()
    from car_price_prediction.utils.common import preprocess_data
    X_train, X_test, y_train, y_test = preprocess_data(
        df=df,
        target_column=preprocessing_config.target_column
    )

    # Base model preparation
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e

[2025-07-12 10:46:31,690: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-12 10:46:31,697: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-12 10:46:31,698: INFO: common: created directory at: artifacts]


AttributeError: 'ConfigurationManager' object has no attribute 'get_data_ingestion_config'